## Utilizaré este archivo para explicar el algoritmo

In [ ]:
board = [4,3]
snake = [[2, 2], [3, 2], [3, 1], [3, 0], [2, 0], [1, 0], [0, 0]]
depth = 3

In [1]:
#Hay que comprobar hacia dónde podemos movernos
def possible_go(board, snake):
    #Creo una lista del tipo [U,D,X,X] donde voy indicando hacia dónde puede realizar el movimiento la serpiente
    #En este caso, podría ir para arriba y para abajo solo. Las X significa que no.
    movements = []
    temp = 0
    
    #MOVIMIENTO HACIA ARRIBA
    head = snake[0]
    #Primero comprobamos si el movimiento estaría dentro de los límites del tablero
    if head[1]-1 < 0:
        temp = 1
    #Ahora comprobamos que no choque consigo misma. El único caso que NO hay que tener en cuenta es la cola
    #Ya que las demás partes si que van a conservar la posición de la siguiente, a excepción de ella.
    for i in range(len(snake)-2):
        if [head[0], head[1]-1] == snake[i+1]:
            temp = 1
    if temp == 1:
        movements.append("X")
    else:
        movements.append("U") 
    temp = 0
    
    #MOVIMIENTO HACIA ABAJO
    if head[1]+1 >= board[1]:
        temp = 1
    for i in range(len(snake)-2):
        if [head[0], head[1]+1] == snake[i+1]:
            temp = 1
    if temp == 1:
        movements.append("X")
    else:
        movements.append("D")
    temp = 0
        
    #MOVIMIENTO HACIA IZQUIERDA
    if head[0]-1 < 0:
        temp = 1
    for i in range(len(snake)-2):
        if [head[0]-1, head[1]] == snake[i+1]:
            temp = 1
    if temp == 1:
        movements.append("X")
    else:
        movements.append("L")
    temp = 0
        
    #MOVIMIENTO HACIA DERECHA
    if head[0]+1 >= board[0]:
        temp = 1
    for i in range(len(snake)-2):
        if [head[0]+1, head[1]] == snake[i+1]:
            temp = 1
    if temp == 1:
        movements.append("X")
    else:
        movements.append("R")
        
    return movements
            

In [2]:
def move(board, snake, direction):
    snake_result = []
    if direction == "U":
        #Resto 1 a la posición 2º de la cabeza, que son las Filas ya que nos movemos en vertical hacia arriba
        snake_result.append([snake[0][0], snake[0][1]-1])
        for i in range(len(snake)-1):
                snake_result.append(snake[i])
    elif direction == "D":
        #Resto 1 a la posición 1º de la cabeza, que son las columnas ya que nos movemos en vertical hacia abajo
        snake_result.append([snake[0][0], snake[0][1]+1])
        for i in range(len(snake)-1):
                snake_result.append(snake[i])
    elif direction == "L":
        #Resto 1 a la posición 1º de la cabeza, que son las columnas ya que nos movemos en horizontal hacia la izquierda
        snake_result.append([snake[0][0]-1, snake[0][1]])
        for i in range(len(snake)-1):
                snake_result.append(snake[i])
    elif direction == "R":
        #Resto 1 a la posición 1º de la cabeza, que son las columnas ya que nos movemos en horizontal hacia la derecha
        snake_result.append([snake[0][0]+1, snake[0][1]])
        for i in range(len(snake)-1):
                snake_result.append(snake[i])
    return snake_result

In [4]:
#He planteado una solución parecida al algoritmo A estrella, es decir, trabajaremos con "nodos".
#La semántica que se ha utilizado ha sido del estilo [0,1,"L"], donde el primer valor es el valor del nodo padre, 
#el segundo es el valor del propio nodo, y el tercero la dirección hacia donde se realiza el movimiento
flag = 0
snake_final = []
positions = []
flag2 = depth
#Tomamos estas listas y variables auxiliares para ir guardando los datos, que después vamos a tener que usar y no queremos perder
id_node = 0
id_parent = 0
nodes = []
snakes_temp = [[snake[:], id_parent]]
final_nodes = []
nodes_temp = []

#Primero compruebo los posibles movimientos desde la posición inicial de la serpiente.

movements = possible_go(board, snake)
for i in movements:
    if i != "X":
        id_node = id_node + 1
        positions.append([id_parent, id_node, i])
        nodes.append([id_parent, id_node, i])
final_nodes.append(nodes)
flag2 = flag2 - 1
    
#Después, haremos un bucle donde vamos a ir buscando los movimientos que puede hacer la serpiente.
#Gracias a la nomeclatura padre-hijo, podemos saber en todo momento y así almacenar, el estado de la serpiente
#para saber desde donde partir en cada caso que tenemos. 
while flag == 0:
    snake_temp = []
    #Recorremos los nodos hijos calculados
    for j in range(len(nodes)):
        #Recorremos los estados de las serpientes que tenemos para tomar la serpiente que corresponda al estado del nodo
        #gracias a que cuando almacenamos el estado de la serpiente, lo almacenamos junto al valor del padre, por lo tanto
        #esto nos indica cuál es el estado del que tenemos que partir en cada nodo.
        for x in range(len(snakes_temp)):
            if snakes_temp[x][1] == nodes[j][0]:
                #Una vez tenemos nuestra posición correcta de la serpiente, realizamos el movimiento indicado en el nodo,
                #y guardamos el nuevo estado de la serpiente, junto al valor correspondiente del id del nodo, para así, 
                #poder acceder a dicho estado cuando lo necesitásemos.
                snake_temp = move(board, snakes_temp[x][0], nodes[j][2])
                snakes_temp.append([snake_temp, nodes[j][1]])
                movements = possible_go(board, snake_temp)
                #Volvemos a calcular los posibles movimientos desde la nueva posición de la serpiente y, como se generan
                #nuevos nodos hijos, los volvemos a almacenar para poder trabajar otra vez desde estos y ver qué posibles
                #movimientos pueden hacer estos nuevos nodos.
                for i in movements:
                    if i != "X":
                        id_node = id_node + 1
                        positions.append([nodes[j][1], id_node, i])
                        nodes_temp.append([nodes[j][1], id_node, i])
    final_nodes.append(nodes_temp)
    nodes = nodes_temp
    nodes_temp = []
    
    flag2 = flag2 - 1
    if flag2 == 0:
        flag = 1
    else:
        snake = snake_temp

In [5]:
#Así, obtenemos una lista, por orden, de los diferentes nodos que hay en cada altura del arbol.
#La primera sublista son los primeros movimientos que puede hacer partiendo del estado inicial.
#Despues, los demás movimientos, se pueden comprobar a través de la nomeclatura padre-hijo, ya que si tenemos
#un nodo [0,1,"L"], y otro nodo [1,3,"L"], vemos que este último tiene como padre el "1", que quiere decir que es un movimiento
#que va después de el nodo, que tiene por id propio "1", es decir, el primero mencionado
final_nodes

[[[0, 1, 'U'], [0, 2, 'L']],
 [[1, 3, 'L'], [2, 4, 'U'], [2, 5, 'L']],
 [[3, 6, 'U'],
  [3, 7, 'D'],
  [3, 8, 'L'],
  [4, 9, 'U'],
  [4, 10, 'L'],
  [4, 11, 'R'],
  [5, 12, 'U']]]

In [6]:
#Como sabemos que la última sublista son todos los diferentes posibles caminos que tiene, recorremos uno a uno hacia arriba
#de la lista, para ir calculando quien es su padre e ir uniendo nodos hasta conseguir las diferentes rutas que puede hacer.
list_final = []
cont = 1
parent = 0
id_node = 0
for i in range(len(final_nodes[depth-1])):
    parent = final_nodes[depth-1][i][0]
    list_final.append([final_nodes[depth-1][i][2]])
    for j in range(depth-1):
        for x in range(len(final_nodes[depth-2-j])):
            if parent == final_nodes[depth-2-j][x][1]:
                list_final[i].insert(0, final_nodes[depth-2-j][x][2])
                parent = final_nodes[depth-2-j][x][0]

In [7]:
list_final

[['U', 'L', 'U'],
 ['U', 'L', 'D'],
 ['U', 'L', 'L'],
 ['L', 'U', 'U'],
 ['L', 'U', 'L'],
 ['L', 'U', 'R'],
 ['L', 'L', 'U']]

In [8]:
#Y obtenemos los posibles caminos 
print("Result:", len(list_final))

Result: 7
